# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

In [3]:
import re
from datetime import datetime

def date_sorter():
    # Patterns to match the different date formats mentioned in the assignment
    date_patterns = [
        r'(?P<month>\d{1,2})[\/\-](?P<day>\d{1,2})[\/\-](?P<year>\d{2,4})',  # MM/DD/YYYY or MM/DD/YY
        r'(?P<month>(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[\.]?[,\-\s]+(?P<day>\d{1,2})(?:st|nd|rd|th)?[,\-\s]+(?P<year>\d{4})',  # Month Day, Year (e.g., Mar 20, 2009)
        r'(?P<day>\d{1,2})(?:st|nd|rd|th)?[,\-\s]+(?P<month>(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[\.]?[,\-\s]+(?P<year>\d{4})',  # Day Month Year (e.g., 20 Mar 2009)
        r'(?P<month>(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[\.]?[,\-\s]+(?P<year>\d{4})',  # Month YYYY (e.g., Mar 2009)
        r'(?P<month>\d{1,2})[\/\-](?P<year>\d{4})',  # MM/YYYY
        r'(?P<year>\d{4})'  # YYYY
    ]
    
    # This will hold the extracted dates and their original indices
    extracted_dates = []
    
    # Iterate over each row of the series
    for i, text in df.iteritems():
        date_found = False
        for pattern in date_patterns:
            match = re.search(pattern, text)
            if match:
                date_dict = match.groupdict()
                
                # Default values
                day = 1
                month = 1
                year = 1900
                
                # Handle year
                if 'year' in date_dict and date_dict['year']:
                    year = int(date_dict['year'])
                    if year < 100:  # Handle two-digit years
                        year += 1900
                
                # Handle month (string months)
                if 'month' in date_dict and date_dict['month']:
                    if date_dict['month'].isalpha():  # If month is a string (e.g., "Mar")
                        month_str = date_dict['month'][:3].capitalize()
                        month = datetime.strptime(month_str, '%b').month
                    else:
                        month = int(date_dict['month'])
                
                # Handle day
                if 'day' in date_dict and date_dict['day']:
                    day = int(date_dict['day'])
                
                # Create a datetime object
                extracted_date = datetime(year, month, day)
                extracted_dates.append((i, extracted_date))
                date_found = True
                break
        
        # In case no date pattern is found, skip
        if not date_found:
            continue
    
    # Sort the dates by the extracted date and tie-break by the original index
    sorted_dates = sorted(extracted_dates, key=lambda x: (x[1], x[0]))
    
    # Return the original indices in the sorted order
    return pd.Series([i for i, _ in sorted_dates])

